# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [ ]:
 @title
from google.colab import userdata
userdata.get('HF_TOKEN')

In [2]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-e4b29vlz
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-e4b29vlz
  Resolved https://github.com/huggingface/accelerate.git to commit 8b4c216b176e7462848d8e6c513ed72d8ea2efe2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-1.13.0.dev0-py3-none-any.whl size=382173 sha256=9da62bfb1ad0771a646cbee79117469da3385251d41f06a0b4555a25951b8005
  Stored in directory: /tmp/pip-ephem-wheel-cache-mzmnyr90/wheels/5a/20/fb/1221fe933b56fe7ac69fd00159d9a1950bc8ced38198abc18f
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.12.0

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

In [4]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [5]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [6]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

This function wraps the call to *model.generate*

In [8]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [9]:
sp_nl2sql2 = """
### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the sample SQL in the ### Examples section to learn more about the database structure**

ADDITIONAL OUTPUT RULES (important):
- Output ONLY the final SQL query (no explanations, no markdown, no code fences/backticks).
- Use ONLY tables and columns that exist in the schema below. Do NOT invent names.
- If the question cannot be answered using the schema, output exactly: I do not know

### Input
Generate a SQL query that answers the question below.
This query will run on a database whose schema is represented in this string:

### Database Schema
CREATE TABLE employees (
  id_usr INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  department TEXT NOT NULL
);

CREATE TABLE salary (
  id_salary INTEGER PRIMARY KEY,
  id_usr INTEGER NOT NULL,
  salary INTEGER NOT NULL,
  pay_date TEXT NOT NULL,
  FOREIGN KEY (id_usr) REFERENCES employees(id_usr)
);

CREATE TABLE studies (
  id_study INTEGER PRIMARY KEY,
  id_usr INTEGER NOT NULL,
  degree TEXT NOT NULL,
  institution TEXT NOT NULL,
  FOREIGN KEY (id_usr) REFERENCES employees(id_usr)
);

### Examples

### Question
List all employee names in the Sales department.
### Response
SELECT name
FROM employees
WHERE department = 'Sales';

### Question
What is the maximum salary?
### Response
SELECT MAX(salary) AS max_salary
FROM salary;

### Question
How many studies does each employee have?
### Response
SELECT e.name, COUNT(s.id_study) AS study_count
FROM employees e
LEFT JOIN studies s ON s.id_usr = e.id_usr
GROUP BY e.id_usr;

### Question
{question}
### Response
"""


In [10]:
sp_nl2sql = sp_nl2sql2.format(question="List all employee names and departments") # check final check on prompt structure.
print(sp_nl2sql)


### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the sample SQL in the ### Examples section to learn more about the database structure**

ADDITIONAL OUTPUT RULES (important):
- Output ONLY the final SQL query (no explanations, no markdown, no code fences/backticks).
- Use ONLY tables and columns that exist in the schema below. Do NOT invent names.
- If the question cannot be answered using the schema, output exactly: I do not know

### Input
Generate a SQL query that answers the question below.
This query will run on a database whose schema is represented in this string:

### Database Schema
CREATE TABLE employees (
  id_usr INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  department TEXT NOT NULL
);

CREATE TABLE salary (
  id_salary INTEGER PRIMARY KEY,
  id_usr INTEGER NOT NULL,
  salary INTEGER N

In [11]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [12]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [13]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the sample SQL in the ### Examples section to learn more about the database structure**

ADDITIONAL OUTPUT RULES (important):
- Output ONLY the final SQL query (no explanations, no markdown, no code fences/backticks).
- Use ONLY tables and columns that exist in the schema below. Do NOT invent names.
- If the question cannot be answered using the schema, output exactly: I do not know

### Input
Generate a SQL query that answers the question below.
This query will run on a database whose schema is represented in this string:

### Database Schema
CREATE TABLE employees (
  id_usr INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  department TEXT NOT NULL
);


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [14]:
sp_nl2sql_openai_shots = """
### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the sample SQL in the ### Examples section to learn more about the database structure**

ADDITIONAL OUTPUT RULES (important):
- Output ONLY the final SQL query (no explanations, no markdown, no code fences/backticks).
- Use ONLY tables and columns that exist in the schema below. Do NOT invent names.
- If the question cannot be answered using the schema, output exactly: I do not know

### Database Schema
CREATE TABLE employees (
  id_usr INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  department TEXT NOT NULL
);

CREATE TABLE salary (
  id_salary INTEGER PRIMARY KEY,
  id_usr INTEGER NOT NULL,
  salary INTEGER NOT NULL,
  pay_date TEXT NOT NULL,
  FOREIGN KEY (id_usr) REFERENCES employees(id_usr)
);

CREATE TABLE studies (
  id_study INTEGER PRIMARY KEY,
  id_usr INTEGER NOT NULL,
  degree TEXT NOT NULL,
  institution TEXT NOT NULL,
  FOREIGN KEY (id_usr) REFERENCES employees(id_usr)
);

### Examples

### Question
List all employee names and departments.
### Response
SELECT name, department
FROM employees;

### Question
What is the maximum salary?
### Response
SELECT MAX(salary) AS max_salary
FROM salary;

### Question
How many studies does each employee have?
### Response
SELECT e.name, COUNT(s.id_study) AS study_count
FROM employees e
LEFT JOIN studies s ON s.id_usr = e.id_usr
GROUP BY e.id_usr;

### Question
{question}
### Response
"""



In [15]:
sp_nl2sql2 = sp_nl2sql2.format(question="Return The name of the best paid employee")
(print(sp_nl2sql2))


### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the sample SQL in the ### Examples section to learn more about the database structure**

ADDITIONAL OUTPUT RULES (important):
- Output ONLY the final SQL query (no explanations, no markdown, no code fences/backticks).
- Use ONLY tables and columns that exist in the schema below. Do NOT invent names.
- If the question cannot be answered using the schema, output exactly: I do not know

### Input
Generate a SQL query that answers the question below.
This query will run on a database whose schema is represented in this string:

### Database Schema
CREATE TABLE employees (
  id_usr INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  department TEXT NOT NULL
);

CREATE TABLE salary (
  id_salary INTEGER PRIMARY KEY,
  id_usr INTEGER NOT NULL,
  salary INTEGER N

In [16]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [17]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the sample SQL in the ### Examples section to learn more about the database structure**

ADDITIONAL OUTPUT RULES (important):
- Output ONLY the final SQL query (no explanations, no markdown, no code fences/backticks).
- Use ONLY tables and columns that exist in the schema below. Do NOT invent names.
- If the question cannot be answered using the schema, output exactly: I do not know

### Input
Generate a SQL query that answers the question below.
This query will run on a database whose schema is represented in this string:

### Database Schema
CREATE TABLE employees (
  id_usr INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  department TEXT NOT NULL
);


The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [18]:
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    ### Database Schema
CREATE TABLE employees (
  id_usr INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  department TEXT NOT NULL
);

CREATE TABLE salary (
  id_salary INTEGER PRIMARY KEY,
  id_usr INTEGER NOT NULL,
  salary INTEGER NOT NULL,
  pay_date TEXT NOT NULL,
  FOREIGN KEY (id_usr) REFERENCES employees(id_usr)
);

CREATE TABLE studies (
  id_study INTEGER PRIMARY KEY,
  id_usr INTEGER NOT NULL,
  degree TEXT NOT NULL,
  institution TEXT NOT NULL,
  FOREIGN KEY (id_usr) REFERENCES employees(id_usr)
);

    ### Samples

Question: List all employee names in the Sales department.
SQL:
SELECT name
FROM employees
WHERE department = 'Sales';

Question: What is the maximum salary?
SQL:
SELECT MAX(salary) AS max_salary
FROM salary;

Question: How many studies does each employee have?
SQL:
SELECT e.name, COUNT(s.id_study) AS study_count
FROM employees e
LEFT JOIN studies s ON s.id_usr = e.id_usr
GROUP BY e.id_usr;

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
"""



In [19]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return The name of the best paid employee")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    ### Database Schema
CREATE TABLE employees (
  id_usr INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  department TEXT NOT NULL
);

CREATE TABLE salary (
  id_salary INTEGER PRIMARY KEY,
  id_usr INTEGER NOT NULL,
  salary INTEGER NOT NULL,
  pay_date TEXT NOT NULL,
  FOREIGN KEY (id_usr) REFERENCES employees(id_usr)
);

CREATE TABLE studies (
  id_study INTEGER PRIMARY KEY,
  id_usr INTEGER NOT NULL,
  degree TEXT NOT NULL,
  institution TEXT NOT NULL,
  FOREIGN KEY (id_usr) REFERENCES employees(id_usr)


In [20]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [21]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT employees.name FROM employees JOIN (SELECT salary.id_usr, MAX(salary) AS max_salary) AS subquery ON employees.id_usr = subquery.id_usr GROUP BY subquery.id_usr ORDER BY subquery.max_salary DESC NULLS LAST LIMIT 1;


#Now the question in spanish.


In [22]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Devuelve el nombre del empleado mejor pagado.") # Best paid employee
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    ### Database Schema
CREATE TABLE employees (
  id_usr INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  department TEXT NOT NULL
);

CREATE TABLE salary (
  id_salary INTEGER PRIMARY KEY,
  id_usr INTEGER NOT NULL,
  salary INTEGER NOT NULL,
  pay_date TEXT NOT NULL,
  FOREIGN KEY (id_usr) REFERENCES employees(id_usr)
);

CREATE TABLE studies (
  id_study INTEGER PRIMARY KEY,
  id_usr INTEGER NOT NULL,
  degree TEXT NOT NULL,
  institution TEXT NOT NULL,
  FOREIGN KEY (id_usr) REFERENCES employees(id_usr)


In [23]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [24]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT name, MAX(salary) AS max_salary FROM salary JOIN employees USING (id_usr) GROUP BY name ORDER BY max_salary DESC NULLS LAST LIMIT 1;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [25]:
print("{question}" in sp_nl2sql, "{question}" in sp_nl2sql2, "{question}" in sp_nl2sql3b) # check before benchmark.


False False True


In [26]:
import re

tests_en = [
    "List all employee names and departments",
    "Return the name of the best paid employee",
    "How many studies does each employee have?",
    "Which manager supervises the most employees?"  # expected: I do not know
]
test_es = "Devuelve el nombre del empleado mejor pagado."

PROMPTS = {
    "NO_SHOTS": sp_nl2sql,          # must still contain {question}
    "OPENAI_SHOTS": sp_nl2sql2,     # must still contain {question}
    "SAMPLE_STYLE_SHOTS": sp_nl2sql3b,  # must still contain {question}
}

def extract_sql(text: str) -> str:
    t = text.strip()
    if "I do not know" in t:
        return "I do not know"

    # fenced block extraction
    m = re.search(r"```(?:sql3|sql)?\s*(.*?)```", t, flags=re.DOTALL | re.IGNORECASE)
    if m:
        return m.group(1).strip()

    # handle opened-but-not-closed fences
    if "```sql3" in t:
        return t.split("```sql3", 1)[-1].strip().replace("```", "").strip()
    if "```sql" in t:
        return t.split("```sql", 1)[-1].strip().replace("```", "").strip()

    return t

def generate_sql(prompt: str, max_new_tokens: int = 180) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda') # Added .to('cuda') here
    outputs = get_outputs(foundation_model, inputs, max_new_tokens=max_new_tokens)
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return extract_sql(decoded)

def run_suite(label: str, template: str, questions):
    print("\n" + "="*80)
    print(label)
    print("="*80)
    for q in questions:
        prompt = template.format(question=q)
        sql = generate_sql(prompt)
        print(f"\nQ: {q}\nSQL:\n{sql}")

# English tests across all prompt variants
for name, tmpl in PROMPTS.items():
    run_suite(name, tmpl, tests_en)

# Spanish test (use OpenAI shots as default)
run_suite("SPANISH_TEST (OPENAI_SHOTS)", PROMPTS["OPENAI_SHOTS"], [test_es])


NO_SHOTS

Q: List all employee names and departments
SQL:
I do not know

Q: Return the name of the best paid employee
SQL:
I do not know

Q: How many studies does each employee have?
SQL:
I do not know

Q: Which manager supervises the most employees?
SQL:
I do not know

OPENAI_SHOTS

Q: List all employee names and departments
SQL:
I do not know

Q: Return the name of the best paid employee
SQL:
I do not know

Q: How many studies does each employee have?
SQL:
I do not know

Q: Which manager supervises the most employees?
SQL:
I do not know

SAMPLE_STYLE_SHOTS

Q: List all employee names and departments
SQL:
SELECT name, department FROM employees;

Q: Return the name of the best paid employee
SQL:
SELECT employees.name FROM employees JOIN (SELECT salary.id_usr, MAX(salary) AS max_salary) AS subquery ON employees.id_usr = subquery.id_usr GROUP BY subquery.id_usr ORDER BY subquery.max_salary DESC LIMIT 1;

Q: How many studies does each employee have?
SQL:
SELECT e.name, COUNT(s.id_study) 

## Exercise Conclusion

- **NO_SHOTS + OPENAI_SHOTS:** returned `I do not know` for every question (even easy ones).  
  →It is possible that the refusal rule dominated, or my output parsing/extraction grabbed the wrong part.

- **SAMPLE_STYLE_SHOTS:** best performance (3/4 English questions produced SQL).  
  - ✅ `SELECT name, department FROM employees;` worked.
  - ⚠️ “Best paid employee” was close but had a small SQL bug (`MAX(salary)` scoping).
  - ✅ “Studies per employee” used correct `LEFT JOIN` + `COUNT`.
  - ❌ Impossible “manager” question hallucinated a `managers` table instead of `I do not know`.

- **Spanish test (OpenAI shots):** returned `I do not know` — likely tied to the same refusal/parsing issue, not necessarily language failure.

